# Машинное обучение для текстов

## Постановка задачи: 
Построить модель классификации коммнетариев пользовавтелей на позитивные и негативные

## Данные:

|text|toxic|
|---|---|
|текст комментария|целевой признак|

## Метрика качества: 
f1

## Этапы работы:

0. Импорт библиотек + Объявление констант

1. Загрузка и подготовка данных

    1.1. целевой признак
    
    1.2. подготовка текста сообщений (приводим к нижнему регистру, удаляем лишние пробелы, проводим токенизацию + лемматизацию, удаляем стоп-слова)
    
2. Обучение моделей

    2.1. делим данные на тренировочну, отложенную, тестовую части
    
    2.2 получаем бейзлайн решение 
    
    2.3. тестирование пайплайнов 
    
    2.4. подбор гиперпараметров
    
## Выводы:
* около 10% комментариев являются токсичными
* модели в порядке убывания метрики качества: SupportVectorClassifier, Lightgbm, RandomForest, LogisticRegression, KNeighborsClassifier
* быстрее всех валидировался SupportVectorClassifier, дольше всех- KNeighborsClassifier
* lightgbm переобучился на тренировочный датасет
* SupportVectorClassifier слабо зависит от гиперпараметров
* результаты валидации

|модель|f1(валидация)|f1(отложенная)|время обучения(сек)|
|---|---|---|---|
|SupportVectorClassifier|0.7638233030317889|0.7672023073753605|2|
|LogisticRegression|0.7200374824206034|0.7250996015936255|3|
|RandomForest|0.7475404652190069|0.7507282563462339|91|
|Lightgbm|0.7369308873299057|0.7216404886561956|25|
|KNeighbors|0.38758200880725563|0.40494092373791624|109|


* финальный результат(SupportVectorClassifier, f1):

|cv|hold|test|
|---|---|---|
|0.76412|0.76449|0.76562|

## Общий вывод:
В процесе выполнения работы была проведена стандартная подготовка текста, с помощью кросс-валидации протестированы различные модели и оптимизирована векторизация текста, лучшее значение f1 было достигнуто с помощью LinearSVC. Цель проекта - обучить модель классифицировать комментарии на позитивные и негативные достигнута.

### 0. Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.sparse import hstack, vstack, csc_matrix
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
import warnings
warnings.filterwarnings('ignore')
import nltk, re, string, gc, pickle
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import KFold, cross_val_score, cross_validate, GridSearchCV, RandomizedSearchCV,\
                                    StratifiedKFold, ParameterGrid
from tqdm import tqdm_notebook
from sklearn.dummy import DummyClassifier
from lightgbm import LGBMClassifier
import time
from sklearn.ensemble import RandomForestClassifier
from collections import defaultdict
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from nltk.corpus import stopwords 

### 0. Константы

In [2]:
# генератор случайных чисел
SEED=13
# доля тестовой части
TEST_SIZE = .2
# доля отложенной части
HOLD_SIZE = .1
# валидация
SKF = StratifiedKFold(3, shuffle = True, random_state = SEED)

### 1. Загрузка и подготовка данных

In [3]:
try:
    # серверный путь
    df= pd.read_csv('/datasets/toxic_comments.csv')
except:
    # локальный путь
    df= pd.read_csv('datasets/toxic_comments.csv')

In [4]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


#### 1.1. целевой признак

In [5]:
print('% токичных комментариев равен {:.0%}'.format(df['toxic'].mean()))

% токичных комментариев равен 10%


#### 1.2. подготовка текста
* приводим к нижнему регистру
* удаляем лишние пробелы
* проводим токенизацию + лемматизацию
* удаляем специальные символы
* удаляем стоп-слова(с помощью параметра в векторайзере tfidf)

In [6]:
# лемматайзер
lemma_function = WordNetLemmatizer()

# словарь с тегами частей речи
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

# проходим по сообщениям
for idx, text in tqdm_notebook(df['text'].items(), total = len(df)):    
    
    # приводим к нижнему регистру, удаляем пунктуацию
    txt = text.lower().strip().translate(str.maketrans('', '', string.punctuation))
    
    # удаляем лишние пробелы
    txt_stripped = ' '.join([element for element in txt.strip().split(' ') if element !=''])
    
    # токенизация + лемматизация
    tokens = word_tokenize(txt_stripped)    
    L_txt_lemmas = []
    for token, tag in pos_tag(tokens):
        lemma = lemma_function.lemmatize(token, tag_map[tag[0]])
        lemma2 = re.sub('[^A-Za-z0-9]+', '', lemma)
        if lemma2 != ' ':
            L_txt_lemmas.append(lemma2)
    txt_lemmas = ' '.join(L_txt_lemmas)
    
    # сохраняем обработанную строку
    df.loc[idx, 'text_preprocessed'] = txt_lemmas

In [7]:
# df = pd.read_pickle('df_text.pkl')

In [11]:
df.sample(10, random_state = SEED)

,text,toxic,text_preprocessed
25680,"(incorrect, moronic allegations of)",1,incorrect moronic allegation of
74202,"As the previous article lead already used, dig...",0,a the previous article lead already use digita...
87912,▲ to ? \n\n...character encoding issues. Oops....,0,to character encode issue oops fix now natura...
130308,Yes. I know that was what Mikka did. And you k...,0,yes i know that be what mikka do and you know ...
147189,Son of a bitchSon of a bitch,1,son of a bitchson of a bitch
39914,"""\n\n Villavar Theory \n\nI strongly suspect t...",0,villavar theory i strongly suspect the reliabi...
112683,Deleting entire paragraphs\n\nAnonymous users ...,0,delete entire paragraph anonymous user and any...
59468,"""\n\nThe trouble with Mahler analysis is that ...",0,the trouble with mahler analysis be that there...
31527,Stop reverting edits because you don't like th...,0,stop revert edits because you dont like them i...
101319,can somebody check my sandbox? I just want to ...,0,can somebody check my sandbox i just want to c...


#### Выводы:
* около 10% комментариев являются токсичными
* произведена предобработка комментариев

### 2. Обучение моделей

#### 2.1. делим данные на тренировочну, отложенную, тестовую части

In [12]:
# признаки, целевой признак
FEATURES, TARGET = df['text_preprocessed'].values.astype('U'), df['toxic'].values

# тренировочная, тестовая выборки
features_tr, features_te, target_tr, target_te = train_test_split(FEATURES,TARGET, test_size=TEST_SIZE,\
                                                                  shuffle = True, random_state = SEED)
# разделитель для отложенной выборки
idx_split = int(round(len(features_tr)*(1-HOLD_SIZE)))

#### 2.2. дамми-модель(бейзлайн)

In [16]:
L_cv_results = []
for strategy in tqdm_notebook(('stratified', 'most_frequent', 'prior', 'uniform')):
    dummy_clf = DummyClassifier(strategy)
    L_cv_results.append(('dummy', strategy,\
                         cross_val_score(dummy_clf,features_tr, target_tr, cv = SKF, scoring = 'f1').mean()))

In [17]:
pd.DataFrame(L_cv_results, columns = ['model', 'features', 'cv_score'])\
  .sort_values('cv_score', ascending = False).reset_index(drop = True)

,model,features,cv_score
0,dummy,uniform,0.169316
1,dummy,stratified,0.104153
2,dummy,most_frequent,0.000000
3,dummy,prior,0.000000


In [18]:
del L_cv_results
gc.collect()

138

#### Выводы:
* получены базовые значения метрик
* лучше всех сработало равномерное предсказание 

#### 2.4. пайплайны (запускать необязательно. результаты сохранил)
* использовались модели разной природы
* признаки: tfidf. max_features(топ n), ngram_range(энграммы)

In [19]:
# модели
models_d = {'svc':LinearSVC(random_state = SEED),\
            'logit': LogisticRegression(random_state = SEED),\
            'rf':RandomForestClassifier(random_state = SEED, n_jobs = -1),\
            'lgb' : LGBMClassifier(random_state = SEED, n_jobs = -1),
            'knn':KNeighborsClassifier()}

In [20]:
# пайплайн №1
ppl = Pipeline([('vectorizer', TfidfVectorizer(stop_words = 'english', max_features = 10000))]) 
ppl.fit(features_tr, target_tr)

# признаки
features1_tr = ppl.transform(features_tr)
features1_te = ppl.transform(features_te)

In [21]:
L1_scores = []
# проходим по моделям
for model_name, model in tqdm_notebook(models_d.items()):
    
    # валидация
    start = time.time()
    mean_cv_score = cross_val_score(model,\
                                    features1_tr[:idx_split], target_tr[:idx_split],\
                                    cv = SKF, scoring = 'f1', n_jobs = -1).mean()
    end = time.time()
    
    # отложенная
    model.fit(features1_tr[:idx_split], target_tr[:idx_split])
    holdout_score = f1_score(target_tr[idx_split:], model.predict(features1_tr[idx_split:]))
    
    L1_scores.append((model_name, mean_cv_score, holdout_score, round(end-start)))

In [25]:
L1_scores_c = [('svc', 0.7638233030317889, 0.7672023073753605, 3),
 ('logit', 0.7200374824206034, 0.7250996015936255, 3),
 ('rf', 0.7475404652190069, 0.7507282563462339, 90),
 ('lgb', 0.7369308873299057, 0.7216404886561956, 22),
 ('knn', 0.38758200880725563, 0.40494092373791624, 99)]


In [23]:
try:    
    cvAB = pd.DataFrame(L1_scores, columns = ['model', 'cv_score', 'holdout_score', 'duration'])\
             .sort_values('cv_score', ascending = False)\
             .set_index('model')
except:
    cvAB = pd.DataFrame(L1_scores_c, columns = ['model', 'cv_score', 'holdout_score', 'duration'])\
             .sort_values('cv_score', ascending = False)\
             .set_index('model')

In [24]:
cvAB

,cv_score,holdout_score,duration
model,,,
svc,0.763823,0.767202,3
rf,0.747540,0.750728,90
lgb,0.736931,0.721640,22
logit,0.720037,0.725100,3
knn,0.387582,0.404941,99


#### Выводы:
* модели в порядке убывания метрики качества: SupportVectorClassifier, Lightgbm, RandomForest, LogisticRegression, KNeighborsClassifier
* быстрее всех валидировался SupportVectorClassifier, дольше всех- KNeighborsClassifier
* lightgbm переобучился на тренировочный датасет

In [25]:
L2_scores = []

# перебираем гиперпараметры tfidf
for ngram_range in tqdm_notebook(((1, 1), (1, 2))):
    for max_features in tqdm_notebook([10**i for i in range(1, 6)]):
        
        # пайплайн №2
        ppl2 = Pipeline([('vectorizer', TfidfVectorizer(stop_words = 'english',\
                                                        max_features = max_features,\
                                                        ngram_range=ngram_range))]) 
        ppl2.fit(features_tr, target_tr)

        # признаки №2
        features2_tr = ppl2.transform(features_tr)
        features2_te = ppl2.transform(features_te)
            
        mean_cv_score = cross_val_score(models_d['svc'],\
                                        features2_tr[:idx_split], target_tr[:idx_split],\
                                        cv = SKF, scoring = 'f1', n_jobs = -1).mean()
        L2_scores.append((ngram_range, max_features, mean_cv_score))

<code style="background:yellow;color:black">сохраненные результаты</code>

In [27]:
L2_scores_с = [((1, 1), 10, 0.0),
 ((1, 1), 100, 0.3430036273771883),
 ((1, 1), 1000, 0.7001445657516144),
 ((1, 1), 10000, 0.7638233030317889),
 ((1, 1), 100000, 0.7620288765035749),
 ((1, 2), 10, 0.0),
 ((1, 2), 100, 0.3427699161249897),
 ((1, 2), 1000, 0.6898170821386255),
 ((1, 2), 10000, 0.7551298004899131),
 ((1, 2), 100000, 0.7581537014948553)]

In [28]:
try:    
    cvAB2 = pd.DataFrame(L2_scores, columns = ['ngram_range', 'max_features', 'mean_cv_score'])\
             .sort_values('mean_cv_score', ascending = False).reset_index(drop = True)             
except:
    cvAB2 = pd.DataFrame(L2_scores_с, columns = ['ngram_range', 'max_features', 'mean_cv_score'])\
             .sort_values('mean_cv_score', ascending = False).reset_index(drop = True)

In [29]:
cvAB2

,ngram_range,max_features,mean_cv_score
0,"(1, 1)",10000,0.763823
1,"(1, 1)",100000,0.762029
2,"(1, 2)",100000,0.758154
3,"(1, 2)",10000,0.755130
4,"(1, 1)",1000,0.700145
5,"(1, 2)",1000,0.689817
6,"(1, 1)",100,0.343004
7,"(1, 2)",100,0.342770
8,"(1, 1)",10,0.000000
9,"(1, 2)",10,0.000000


In [30]:
best_score2 = cvAB2['mean_cv_score'].max()
for max_features in tqdm_notebook(np.arange(5000, 100000, 10000)):
    
    # пайплайн №3
    ppl3 = Pipeline([('vectorizer', TfidfVectorizer(stop_words = 'english',\
                                                    max_features = max_features,\
                                                    ngram_range=(1, 1)))]) 
    ppl3.fit(features_tr, target_tr)

    # признаки №3
    features3_tr = ppl3.transform(features_tr)
    features3_te = ppl3.transform(features_te)
    
    mean_cv_score = cross_val_score(models_d['svc'],\
                                    features3_tr[:idx_split], target_tr[:idx_split],\
                                    cv = SKF, scoring = 'f1', n_jobs = -1).mean()
    if mean_cv_score>best_score2:
        best_score2 = mean_cv_score
        best_max_features = max_features     

In [31]:
n_iterations = 10
for i in tqdm_notebook(range(n_iterations)):
    np.random.RandomState(i)
    max_features = np.random.choice(np.arange(best_max_features-5000,best_max_features+5000))
    
    # пайплайн №4
    ppl4 = Pipeline([('vectorizer', TfidfVectorizer(stop_words = 'english',\
                                                    max_features = max_features,\
                                                    ngram_range=(1, 1)))]) 
    ppl4.fit(features_tr, target_tr)

    # признаки №4
    features4_tr = ppl4.transform(features_tr)
    features4_te = ppl4.transform(features_te)
    
    mean_cv_score = cross_val_score(models_d['svc'],\
                                    features4_tr[:idx_split], target_tr[:idx_split],\
                                    cv = SKF, scoring = 'f1', n_jobs = -1).mean()
    if mean_cv_score>best_score2:
        best_score2 = mean_cv_score
        best_features_tr = features4_tr
        best_features_te = features4_te
    else:
        best_features_tr = features3_tr
        best_features_te = features3_te

#### 2.5. подбор гиперпараметров

In [32]:
pg = {'C': [.1, 1, 5, 10, 100], 'penalty':['l1', 'l2', None], 'random_state':[SEED]}
gs = GridSearchCV(models_d['svc'], param_grid = pg, cv =  SKF, scoring = 'f1', n_jobs=-1, verbose = 1)
gs.fit(best_features_tr[:idx_split], target_tr[:idx_split])

print('f1 cv (best) = {:.5f}'.format(gs.best_score_))
print('f1 hold (best) = {:.5f}'.format(f1_score(target_tr[idx_split:],\
                                                gs.best_estimator_.predict(best_features_tr[idx_split:]))))
print('f1 test (best) = {:.5f}'.format(f1_score(target_te,\
                                                gs.best_estimator_.predict(best_features_te))))

Fitting 3 folds for each of 15 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  45 | elapsed:    1.1s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    8.2s finished


f1 cv (best) = 0.76412
f1 hold (best) = 0.76449
f1 test (best) = 0.76562


#### Выводы:
* linearSVC слабо зависит от гиперпараметров